In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import fastai
from fastai.vision import *
from fastai.callbacks import *
from torchvision.models import vgg16_bn
from torchvision import datasets, models, transforms
from torch.optim import lr_scheduler
import time
from tqdm import tqdm, tqdm_notebook
import copy
import PIL

In [3]:
path = Path('./data/crappified/dataset_1/')
path_hr = path/'orig_spectr'

In [4]:
meta_spectr= pd.read_csv('./data/crappified/dataset_1/meta/meta_spectr.csv')
meta_spectr.subset = meta_spectr.subset == 'valid'

In [5]:
vgg_m = vgg16_bn(True).features.cuda().eval()
requires_grad(vgg_m, False)

In [6]:
vgg_orig = vgg16_bn(True)

In [7]:
new_head = torch.nn.Sequential(*(list(vgg_orig.classifier.children())[:-1] + [torch.nn.Linear(in_features=4096, out_features=8, bias=True)]))
new_head

Sequential(
  (0): Linear(in_features=25088, out_features=4096, bias=True)
  (1): ReLU(inplace)
  (2): Dropout(p=0.5)
  (3): Linear(in_features=4096, out_features=4096, bias=True)
  (4): ReLU(inplace)
  (5): Dropout(p=0.5)
  (6): Linear(in_features=4096, out_features=8, bias=True)
)

In [8]:
vgg_orig.classifier = new_head

In [9]:
class Dataset_from_df(Dataset):
    
    def __init__(self, df_meta, path_img, loader, transform=None):
        self.path_img = Path(path_img)
        self.data = df_meta.reset_index(drop=True)
        self.transform = transform
        self.loader = loader
        self.classes = list(np.unique(df_meta.genre))
        class_to_idx = {self.classes[i]: i for i in range(len(self.classes))}
        self.data.genre = self.data.genre.apply(lambda x: class_to_idx[x])
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        image = self.loader(self.path_img / self.data.loc[index, 'spectr_id'])
        label = self.data.loc[index, 'genre']
        if self.transform is not None:
            image = self.transform(image)
            
        return image, label

In [10]:
df_meta = pd.read_csv('./data/crappified/dataset_1/meta/meta_spectr.csv')
df_meta.head()

,spectr_id,genre,subset
0,2_0.png,Hip-Hop,train
1,2_1.png,Hip-Hop,train
2,2_2.png,Hip-Hop,train
3,2_3.png,Hip-Hop,train
4,2_4.png,Hip-Hop,train


In [11]:
from PIL import Image

def pil_loader(path):
    # open path as file to avoid ResourceWarning (https://github.com/python-pillow/Pillow/issues/835)
    with open(path, 'rb') as f:
        img = PIL.Image.open(f)
        return img.convert('RGB')

def default_loader(path):
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader(path)
    else:
        return pil_loader(path)
    
TrainDataLoader = Dataset_from_df(df_meta[df_meta.subset == 'train'], path_img = './data/crappified/dataset_1/orig_spectr/', loader = default_loader)

In [12]:
TrainDataLoader[0]

(<PIL.Image.Image image mode=RGB size=128x128 at 0x7F87F10F6048>, 3)

In [13]:

data_dir = './data/crappified/dataset_1/orig_spectr/'
TRAIN = 'train'
VAL = 'valid'

# VGG-16 Takes 224x224 images as input, so we resize all of them
data_transforms = {
    TRAIN: transforms.Compose([
        transforms.Resize(256),
        # Data augmentation is a good practice for the train set
        # Here, we randomly crop the image to 224x224 and
        # randomly flip it horizontally. 
        transforms.RandomResizedCrop(224),
        transforms.ToTensor(),
    ]),
    VAL: transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
    ])
}

image_datasets = {
    x: Dataset_from_df(df_meta[df_meta.subset == x],
                       path_img = data_dir, loader = default_loader,
                       transform=data_transforms[x]
                      )
    for x in [TRAIN, VAL]
}

dataloaders = {
    x: torch.utils.data.DataLoader(
        image_datasets[x], batch_size=8,
        shuffle=True, num_workers=4
    )
    for x in [TRAIN, VAL]
}

dataset_sizes = {x: len(image_datasets[x]) for x in [TRAIN, VAL]}

for x in [TRAIN, VAL]:
    print("Loaded {} images under {}".format(dataset_sizes[x], x))
    
print("Classes: ")
class_names = image_datasets[TRAIN].classes
print(image_datasets[TRAIN].classes)

Loaded 527073 images under train
Loaded 130750 images under valid
Classes: 
['Electronic', 'Experimental', 'Folk', 'Hip-Hop', 'Instrumental', 'International', 'Pop', 'Rock']


In [14]:
for a,b in dataloaders['train']:
    i, l = a, b
    break

In [15]:
image_datasets['train'][0]

(tensor([[[0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          [0.0039, 0.0039, 0.0039,  ..., 0.0039, 0.0039, 0.0039],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0118, 0.0118, 0.0118,  ..., 0.0039, 0.0039, 0.0039],
          [0.0118, 0.0118, 0.0118,  ..., 0.0039, 0.0039, 0.0039],
          [0.0118, 0.0118, 0.0118,  ..., 0.0039, 0.0039, 0.0039],
          ...,
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],
 
         [[0.0078, 0.0078, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0078, 0.0078, 0.0078,  ..., 0.0039, 0.0039, 0.0039],
          [0.0078, 0.0078, 0.0078,  ...,

In [16]:
device='cuda'
def train_model(model, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'valid']:
            if phase == 'train':
                scheduler.step()
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in tqdm(dataloaders[phase]):
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, preds = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model

In [17]:
model_ft = vgg_orig.cuda()
criterion = nn.CrossEntropyLoss()
# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)
# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)

In [18]:
model_ft = train_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler,
                       num_epochs=1)

  0%|          | 0/65885 [00:00<?, ?it/s]

Epoch 0/0
----------


  0%|          | 0/16344 [00:00<?, ?it/s]

train Loss: 1.6886 Acc: 0.3846


100%|██████████| 16344/16344 [23:29<00:00, 11.59it/s]

valid Loss: 1.5298 Acc: 0.4698

Training complete in 253m 32s
Best val Acc: 0.000000


In [20]:
torch.save(model_ft, './data/crappified/dataset_1/model_clas/model_1.pth')

In [21]:
torch.save(model_ft.state_dict(), './data/crappified/dataset_1/model_clas/model_1_weights.pth')